# Classificação multinomial

### Índice <a name="topo"></a>
- 1. [Introdução](#1)
    - 1.1 [Carregar a base do exemplo](#1.1)
    - 1.2 [Dados faltantes](#1.2)
    - 1.3 [Descritiva inicial](#1.3)
- 2. [Preenchimento de dados faltantes](#2)
    - 2.1 [2.1 Análise descritiva: vale a pena preencher ```sex``` com modelo?](#2.1)
    - 2.2 [2.2 Modelo de preenchimento](#2.2)
- 3. [Classificação multinomial](#3)
    - 3.1 [A árvore](#3.1)
    - 3.2 [Pós-poda](#3.2)
    - 3.3 [A árvore](#3.3)
    - 3.4 [Avaliação do resultado](#3.4)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

### 1. Introdução <a name="1"></a>
[Voltar para o índice](#topo)

#### 1.1 Carregando a base de exemplo<a name="1.1"></a>
[Voltar para o índice](#topo)

Nesta aula vamos trabalhar com a base *penguins*. Essa base pode ser carregada com a ajuda do pacote ```seaborn``` conforme abaixo. Já trabalhamos algumas vezes com ela, ela traz dados biométricos de três raças de pinguins. A nossa tarefa vai ser escrever um algoritmo que classifique o pinguim em uma dessas espécies com base nas características biométricas dele.

In [ ]:
pg = sns.load_dataset('penguins')
pg.shape

In [ ]:
pg

#### 1.2 Dados faltantes<a name="1.2"></a>
[Voltar para o índice](#topo)  

Vamos trabalhar com o *scikitlearn*, o que significa que não podemos ter dados *missing* nem em formato *não numérico* na base. Conforme a análise abaixo, podemos ver que há dois valores missing em quatro variáveis, e a variável ```sex``` possui 11 valores não preenchidos.

In [ ]:
pg.isna().sum()

#### 1.3 Descritiva inicial<a name="1.3"></a>
[Voltar para o índice](#topo)  

Vamos fazer uma matriz de dispersão (*scatterplot matrix*) para visualizar quão promissora é a classificação das raças de pinguim de acordo com as variáveis biométricas.

In [ ]:
sns.pairplot(pg, hue='species');

Vamos nos certificar de que essas duas observações são o mesmo registro. Se for o caso, vai ter baixo impacto no nosso algoritmo elimina-los.

In [ ]:
pg[pg['bill_depth_mm'].isna()]

Sim, são as mesmas, então eliminar estas observações implica em perder apenas duas observações (e bem incompletas por sinal).

In [ ]:
pg1 = pg[~pg['bill_depth_mm'].isna()].copy()

### 2 Preenchimento de dados faltantes<a name="2"></a>
[Voltar para o índice](#topo)  

Para a variável ```sex```, temos mais 9 registros com dados faltantes. Claro que em se tratando de análise de dados, podemos ter inúmeras alternativas, mas eu vejo aqui três razoáveis com prós e contras:

- Eliminar a variável ```sex```. Essa alternativa pode significar abrir mão de uma fonte de varição importante que deve ajdar a classificar os indivíduos em suas respectivas classes.
- Eliminar os 9 registros faltantes: parece uma solução ok, mas já vamos para 11 registros eliminados de 344.
- Fazer imputação de dados: Isso significa tentar "adivinhar" qual o valor para a variável faltante para não perder o registro. Se fizermos isso mal, podemos introduzir ruído na base, por outro lado, recuperamos esses registros.

A terceira opção parece mais interessante. Agora... que método podemos utilizar? 
- Podemos fazer imputação pela moda - utilizar o valor que mais aparece na base.
- Imputação aleatória: simplemente sortear um valor conforme a distribuição observada
- Ou ainda, a solução que vamos adotar, fazer uma "imputação por árvore":
    - Vamos construir uma árvore de decisão para classificar o sexo dos pinguins dadas as demais variáveis
    - Em seguida vamos usar esta árvore para classificar os valores faltantes.

Se a nossa árvore for boa, vamos cometer um erro pequeno, e em todo caso, vamos "salvar" as observações com dados faltantes. Como contra, eu apontaria o trabalho de se construir esta árvore. No caso, como tudo é um aprendizado, vamos arregaçar as mangas :)

#### 2.1 Análise descritiva: vale a pena preencher ```sex``` com modelo?<a name="2.1"></a>
[Voltar para o índice](#topo)  

Em primeiro lugar, precisamos tentar entender se "vale a pena" fazer essa árvore. Intuitivamente, entendemos que os pinguins devem ter características bem diferentes no macho e na fêmea, a começar do peso. Vamos ver se os dados indicam isto:

In [ ]:
sns.set_theme(style="ticks")

sns.boxplot(data=pg, x='species', hue="sex", y='bill_length_mm')

Pelo visto sim, machos e fêmeas possuem medidas bem diferentes dada a espécie, principalmente.

#### 2.2 Modelo de preenchimento<a name="2.2"></a>
[Voltar para o índice](#topo)  

Vamos então para o modelo de imputação de dados. O primeiro passo é **preparar a base para classificar sexo**. Não vamos perder o objetivo de vista: agora queremos fazer um modelo para prever sexo, para imputar os valores faltantes. Vamos lá:

- Vamos retirar a variável ```sex``` da base
- Utilizar o ```get_dummies()``` para criar as variáveis dummy de ```species``` e ```island```
- Dividir a base em treino e teste
- Fazer a pós poda pelo ccp-alpha
- Classificar a base original para não perder as 9 observações

Sim, dá um trabalhinho, mas com o tempo isso tudo vai ficando mais automático ;)

Pois é, o nosso objetivo principal aqui é **aprender**. Caso contrário, seria absolutamente legítimo você decidir não fazer isto porque acha que não vale o custo vs benefício - aliás, esse é um julgamento muito útil para o cientista de dados ;)


##### Eliminando *missings* para a construção do modelo

In [ ]:
pg2 = pg1[~pg1['sex'].isna()]
pg2.shape

In [ ]:
pg2.isna().sum()

##### Criando *dummies* e separando bases de treino e teste

In [ ]:
y0 = pg2.sex
X0 = pd.get_dummies(pg2.drop(columns = ['sex']), drop_first=True)

X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0)
X0.head()

##### Construindo a árvore para classificar ```sex```
A célula abaixo monta a árvore, treina "deixando ela ser feliz" (sem limite de profundidade, número de observações ou complexidade). Também salva os *ccp alpahs* para fazermos a pós poda.

In [ ]:
clf0 = DecisionTreeClassifier(random_state=42)
path = clf0.cost_complexity_pruning_path(X0_train, y0_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

##### Realiza a pós poda
Sobre a árvore acima, que é super complexa, vamos obter árvores derivadas e menos complexas seguindo o caminho indicado pelo custo de complexidade (o *ccp alpha*). Para cada árvore derivada, vamos avaliar a métrica de desempenho (no caso, a acurácia), e escolher a melhor árvore para fazer a imputação. 

In [ ]:
clfs0 = []
for ccp_alpha in ccp_alphas:
    clf0 = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(X0_train, y0_train)
    clfs0.append(clf0)

Vamos montar abaixo um gráfico que já vimos na aula de árvores binárias: para cada poda indicada pelo *ccp*, colocamos a métrica de desempenho da árvore na base de desenvolvimento e na base de testes.

In [ ]:
train_scores = [clf0.score(X0_train, y0_train) for clf0 in clfs0]
test_scores  = [clf0.score(X0_test, y0_test)   for clf0 in clfs0]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("Acurácia")
ax.set_title("Acurácia x alpha do conjunto de dados de treino e teste")
ax.plot(ccp_alphas, train_scores, marker='o', label="treino",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="teste",
        drawstyle="steps-post")
ax.legend()
plt.show()

Pronto. Abaixo tem uma "firula": Eu quero a melhor árvore, sim, mas se houverem empates, a que tenha a menor complexidade. Isso significa que eu quero pegar a maior métrica de desempenho no objeto ```test_scores```, mas se houver empates, queremos a primeira ocorrência desse máximo **da direita para a esquerda**.

Esse é um ótimo quebra cabeças de Python :) Abaixo temos um código bem pythonico que resolve isso.

In [ ]:
ind_melhor_arvore = len(test_scores) - test_scores[::-1].index(max(test_scores)) - 1
melhor_arvore = clfs0[ind_melhor_arvore]

##### Substituindo os missings

Agora que fizemos a árvore para a variável ```sex```, vamos utilizar esta regra para substituir os valores missing na base original. Aí você pode me perguntar: "Por quê não fazemos para as outras 4 variávels?" E eu respondo com outra pergunta: "Séro? Você acha mesmo que vale a pena por 2 observações e sem ter muito com o que prever? Boa sorte então, pode fazer!" Repito: sempre vale refletir se vale o esforço de se construir um modelo.

In [ ]:
# Seleciona as variáveis a serem substituídas
prever = pd.get_dummies(pg1.drop(columns = ['sex']), drop_first=True)
prever_sex = prever[pg1['sex'].isna()]

#faz a classificação conforme a árvore
inputação_sex = melhor_arvore.predict(prever_sex)
inputação_sex

In [ ]:
# Faz a substituição propriamente dita
pg1.loc[pg1['sex'].isna(), 'sex'] = inputação_sex

print(pg1.shape)
pg1.isna().sum()

Vamos salvar esta base de dados tratada, para uso futuro.

In [ ]:
pg1.to_csv('pg1.csv', index=False)

### 3. Classificação multinomial<a name="3"></a>
[Voltar para o índice](#topo)  

Pronto, agora sim vamos para o tema principal: construir um algoritmo que classifique a **espécie** do pinguim em uma das três que aparecem na base de dados. A mecânica aqui é MUITO parecida com a classificação binária, basta utilizarmos generalizações das funções de impureza que comportem mais de duas classes e pronto: podemos usar o mesmo algoritmo.

Vamos seguir aqui passos bem parecidos:
- Tratar *missings* (acabamos de fazer isso)
- Criar *dummies*
- Construir a árvore (e deixa-la "ser feliz")
- Realizar a pós-poda e obter a árvore "ótima"

In [67]:
pg1.shape

In [68]:
X = pd.get_dummies(pg1.drop(columns = ['species', 'island']), drop_first=True)
y = pg1['species']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [69]:
y_train.value_counts()

#### 3.1 A árvore<a name="3.1"></a>
[Voltar para o índice](#topo)

A árvore é bem semelhante à que já conhecemos, mas com mais de uma alternativa de resposta.

In [70]:
clf = DecisionTreeClassifier(random_state=2360873, max_depth=3).fit(X_train, y_train)

In [71]:
import graphviz
from sklearn.tree import export_graphviz
# DOT data
dot_data = export_graphviz(clf, out_file=None, 
                                feature_names=X_train.columns,  
                                class_names=['Adelie', 'Chinstrap', 'Gentoo'],
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph

# Decision Tree Classifier from scikit-learn sklearn visualized with graphviz in Python

Avaliando a árvore completa

#### 3.2 Pós-poda<a name="3.2"></a>
[Voltar para o índice](#topo)  

Essa etapa é bem parecida com o que já fizemos. Vamos observar os "caminhos de poda" da árvore "grande", avaliar uma métrica de qualidade do modelo na base de testes, e selecionar a melhor.

In [72]:
caminho = DecisionTreeClassifier(random_state=2360873).cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = caminho.ccp_alphas, path.impurities

In [73]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(X_train, y_train)
    clfs.append(clf)

In [74]:
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("Acurácia")
ax.set_title("Acurácia x alpha do conjunto de dados de treino e teste")
ax.plot(ccp_alphas, train_scores, marker='o', label="treino",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="teste",
        drawstyle="steps-post")
ax.legend()
plt.show()

#### 3.3 Selecionando a melhor árvore<a name="3.3"></a>
[Voltar para o índice](#topo)  

Vamos pegar a melhor árvore utilizando a mesma lógica da árvore que usamos para preencher os registros faltantes na variável ```sex```.

In [75]:
ind_melhor_arvore = len(test_scores) - test_scores[::-1].index(max(test_scores)) - 1
melhor_arvore = clfs[ind_melhor_arvore]
melhor_arvore

#### 3.4 Avaliando o resultado<a name="3.3"></a>
[Voltar para o índice](#topo)  

Agora vamos avaliar o resultado da nossa classificação utilizanod a acrurácia, na base de testes. Para um visual um pouco melhor, vamos utilizar a matriz de confusão.

In [76]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(melhor_arvore, X_test, y_test);

Sensacional esse resultado não? A taxa de acertos é impressionante! Até era de se esperar algo bom pela análise descritiva.

Mas, para refletir, o gancho para a próxima aula:

Será que "demos sorte" de a base de testes ter esse desempenho?  
Com outra base teriamos o mesmo desempenho?  
Como podemos obter uma métrica mais "confiável" do desempenho desse algoritmo?